In [1]:
cd /home/urwa/Documents/Projects/NYU Remote/UrbanTrafficPrediction/

/home/urwa/Documents/Projects/NYU Remote/UrbanTrafficPrediction


In [2]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [3]:
from utils.lstm_utils import prepare_data_lstm, lstm_monthly_dataloader
from utils.lstm_utils import get_device

from utils.lstm_utils import evaluate_edge_monthy
from models.models import LSTM

from utils.lstm_utils import train_one_epoch, store_chekpoint, load_chekpoint
from utils.lstm_utils import evaluate_lstm_pipeline_model, run_inference

In [4]:
torch.manual_seed(2020)
np.random.seed(2020)

In [5]:
config = {'lr': 0.00034439316653688684,
 'layers': 3,
 'step_size': 11,
 'gamma': 0.761795969995615,
 'bptt': 19,
 'dropout': 0.1227497445640586}

In [6]:
device = get_device(cuda=True)
device

device(type='cuda')

In [7]:
data_path = '/home/urwa/Documents/Projects/NYU Remote/data/featureData/com_lga.csv'
weights_path = '/home/urwa/Documents/Projects/NYU Remote/data/featureData/com_lga_weights.csv'
test_data_path='/home/urwa/Documents/Projects/NYU Remote/data/featureData/lga.csv'

exp_dir = '/home/urwa/Documents/Projects/NYU Remote/UrbanTrafficPrediction/data/lstm_12fold/lga'

In [8]:
dataset, targetColumns, features_cols = prepare_data_lstm(data_path)

Raw Shape:  (8757, 113)
Cleaned Shape:  (8757, 38)
Target columns: 24
Feature coumns:  13


In [10]:
bptt = config['bptt']

R2List = []
EdgeR2List = []
residual_list = []


for m in range(1,13):
    
    print('-------------------------------------------------')
    print('-------------------------------------------------')
    print("Month: ", m)

    train_inout_seq, test_inout_seq = lstm_monthly_dataloader(dataset,features_cols, targetColumns, m,
                                                              bptt, device)
    
    lstm_layers = config['layers']
    network_size = len(targetColumns)
    feat_size = len(features_cols)
    dropout = config['dropout']
    hidden_layer_size=100
    
    
    model = LSTM(feat_size, network_size, hidden_layer_size, lstm_layers, dropout).to(device)
    loss_function = nn.L1Loss()   
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=config['step_size'], gamma=config['gamma']) 
    
    
    
    epochs = 80
    if m == 7:
        start_epoch=62
        best_r2=0.903
        model,optimizer,scheduler = load_chekpoint(exp_dir, model,optimizer,scheduler)
        
    else:
        start_epoch=0
        best_r2=0
        
    
    print("\n Training model...")
    for i in range(start_epoch,epochs):
        
        loss = train_one_epoch(model, optimizer, loss_function, train_inout_seq, device)
        scheduler.step()
        store_chekpoint(exp_dir, model, optimizer, scheduler)

        residual, r2, rmse, mae = evaluate_lstm_pipeline_model(model, test_inout_seq, device)
        print(f'epoch: {i:3} loss: {loss:10.8f} r2: {r2:5.3f} rmse: {rmse:5.3f} mae: {mae:5.3f}')

        if r2 > best_r2:
            best_r2 = r2
            best_residual = residual
            torch.save(model.state_dict(), os.path.join(exp_dir, str(m)+'.pt'))
            np.save(os.path.join(exp_dir,str(m)+'.npy'), best_residual)
            edge_res, edge_r2, edge_rmse, edge_mae = evaluate_edge_monthy(model, test_inout_seq, device, 
                                                                   targetColumns, weights_path, 
                                                              test_data_path, m, bptt)
            print('Saving model and residual')
            print('edge r2: ', edge_r2)
            
            
    print("best_r2: ", best_r2)

    R2List.append(best_r2)
    EdgeR2List.append(edge_r2)
    residual_list.append(best_residual)

-------------------------------------------------
-------------------------------------------------
Month:  8
train test split
train shape:  (8013, 38)
test shape:  (744, 38)
train feature tensor shape : torch.Size([8013, 13])
train target tensor shape : torch.Size([8013, 24])
test feature tensor shape : torch.Size([744, 13])
test target tensor shape : torch.Size([744, 24])

sequences
torch.Size([19, 13]) torch.Size([19, 24]) torch.Size([19, 24])

 Training model...
------- Saving checkpoint---------------
epoch:   0 loss: 11.46419651 r2: 0.685 rmse: 374.154 mae: 7.036
Saving model and residual
edge r2:  0.5091489545685565
------- Saving checkpoint---------------
epoch:   1 loss: 6.35121240 r2: 0.841 rmse: 189.153 mae: 5.964
Saving model and residual
edge r2:  0.6309760094625355
------- Saving checkpoint---------------
epoch:   2 loss: 5.39911322 r2: 0.879 rmse: 143.876 mae: 5.435
Saving model and residual
edge r2:  0.662097662354337
------- Saving checkpoint---------------
epoch:   3 

epoch:  70 loss: 3.05003030 r2: 0.884 rmse: 137.678 mae: 5.175
------- Saving checkpoint---------------
epoch:  71 loss: 3.04609976 r2: 0.885 rmse: 136.781 mae: 5.160
------- Saving checkpoint---------------
epoch:  72 loss: 3.04085895 r2: 0.884 rmse: 137.601 mae: 5.184
------- Saving checkpoint---------------
epoch:  73 loss: 3.04023017 r2: 0.885 rmse: 136.561 mae: 5.166
------- Saving checkpoint---------------
epoch:  74 loss: 3.03823511 r2: 0.883 rmse: 138.398 mae: 5.185
------- Saving checkpoint---------------
epoch:  75 loss: 3.03307509 r2: 0.884 rmse: 137.860 mae: 5.175
------- Saving checkpoint---------------
epoch:  76 loss: 3.02990132 r2: 0.883 rmse: 138.787 mae: 5.189
------- Saving checkpoint---------------
epoch:  77 loss: 3.02832082 r2: 0.884 rmse: 137.833 mae: 5.173
------- Saving checkpoint---------------
epoch:  78 loss: 3.01828952 r2: 0.884 rmse: 137.402 mae: 5.180
------- Saving checkpoint---------------
epoch:  79 loss: 3.01743999 r2: 0.885 rmse: 136.216 mae: 5.170
b

------- Saving checkpoint---------------
epoch:  61 loss: 3.08814706 r2: 0.910 rmse: 137.007 mae: 4.956
------- Saving checkpoint---------------
epoch:  62 loss: 3.08463729 r2: 0.910 rmse: 136.876 mae: 4.967
------- Saving checkpoint---------------
epoch:  63 loss: 3.08318172 r2: 0.910 rmse: 136.735 mae: 4.951
------- Saving checkpoint---------------
epoch:  64 loss: 3.08063655 r2: 0.911 rmse: 135.939 mae: 4.934
------- Saving checkpoint---------------
epoch:  65 loss: 3.07611724 r2: 0.909 rmse: 138.292 mae: 4.975
------- Saving checkpoint---------------
epoch:  66 loss: 3.07393785 r2: 0.909 rmse: 137.880 mae: 4.964
------- Saving checkpoint---------------
epoch:  67 loss: 3.06035982 r2: 0.910 rmse: 136.657 mae: 4.960
------- Saving checkpoint---------------
epoch:  68 loss: 3.05671007 r2: 0.908 rmse: 139.802 mae: 5.000
------- Saving checkpoint---------------
epoch:  69 loss: 3.05682461 r2: 0.909 rmse: 138.797 mae: 4.987
------- Saving checkpoint---------------
epoch:  70 loss: 3.0505

------- Saving checkpoint---------------
epoch:  51 loss: 3.15715457 r2: 0.922 rmse: 139.794 mae: 4.996
------- Saving checkpoint---------------
epoch:  52 loss: 3.15397849 r2: 0.921 rmse: 142.008 mae: 5.031
------- Saving checkpoint---------------
epoch:  53 loss: 3.15096164 r2: 0.921 rmse: 140.984 mae: 5.017
------- Saving checkpoint---------------
epoch:  54 loss: 3.14503572 r2: 0.922 rmse: 139.714 mae: 5.009
------- Saving checkpoint---------------
epoch:  55 loss: 3.13881383 r2: 0.920 rmse: 143.485 mae: 5.023
------- Saving checkpoint---------------
epoch:  56 loss: 3.12403188 r2: 0.920 rmse: 142.984 mae: 5.032
------- Saving checkpoint---------------
epoch:  57 loss: 3.11819495 r2: 0.921 rmse: 142.045 mae: 5.028
------- Saving checkpoint---------------
epoch:  58 loss: 3.11291173 r2: 0.920 rmse: 143.303 mae: 5.047
------- Saving checkpoint---------------
epoch:  59 loss: 3.10664736 r2: 0.921 rmse: 142.182 mae: 5.012
------- Saving checkpoint---------------
epoch:  60 loss: 3.1042

epoch:  40 loss: 3.22621849 r2: 0.879 rmse: 230.545 mae: 6.063
------- Saving checkpoint---------------
epoch:  41 loss: 3.21996138 r2: 0.877 rmse: 234.768 mae: 6.094
------- Saving checkpoint---------------
epoch:  42 loss: 3.20899620 r2: 0.879 rmse: 231.971 mae: 6.084
------- Saving checkpoint---------------
epoch:  43 loss: 3.20226789 r2: 0.878 rmse: 233.465 mae: 6.092
------- Saving checkpoint---------------
epoch:  44 loss: 3.19329205 r2: 0.882 rmse: 225.783 mae: 6.036
------- Saving checkpoint---------------
epoch:  45 loss: 3.16819221 r2: 0.879 rmse: 230.559 mae: 6.086
------- Saving checkpoint---------------
epoch:  46 loss: 3.16458509 r2: 0.881 rmse: 227.031 mae: 6.051
------- Saving checkpoint---------------
epoch:  47 loss: 3.15838517 r2: 0.880 rmse: 228.689 mae: 6.061
------- Saving checkpoint---------------
epoch:  48 loss: 3.15422858 r2: 0.879 rmse: 230.528 mae: 6.034
------- Saving checkpoint---------------
epoch:  49 loss: 3.14827183 r2: 0.880 rmse: 228.908 mae: 6.064
-

------- Saving checkpoint---------------
epoch:  29 loss: 3.44238095 r2: 0.887 rmse: 208.135 mae: 5.686
------- Saving checkpoint---------------
epoch:  30 loss: 3.42584311 r2: 0.891 rmse: 200.779 mae: 5.628
------- Saving checkpoint---------------
epoch:  31 loss: 3.41320369 r2: 0.894 rmse: 195.612 mae: 5.590
------- Saving checkpoint---------------
epoch:  32 loss: 3.39784348 r2: 0.893 rmse: 196.879 mae: 5.635
------- Saving checkpoint---------------
epoch:  33 loss: 3.38344767 r2: 0.883 rmse: 215.059 mae: 5.700
------- Saving checkpoint---------------
epoch:  34 loss: 3.34683253 r2: 0.885 rmse: 212.063 mae: 5.721
------- Saving checkpoint---------------
epoch:  35 loss: 3.33659084 r2: 0.887 rmse: 207.485 mae: 5.698
------- Saving checkpoint---------------
epoch:  36 loss: 3.32664313 r2: 0.885 rmse: 212.838 mae: 5.719
------- Saving checkpoint---------------
epoch:  37 loss: 3.31533088 r2: 0.887 rmse: 207.608 mae: 5.712
------- Saving checkpoint---------------
epoch:  38 loss: 3.3066